In [22]:
import torch

In [23]:
dim = 6
end = 10
theta = 1000

In [24]:
freqs = 1.0 / (theta ** (torch.arange(0, dim, 2)[: (dim // 2)].float() / dim))
print(freqs)
t = torch.arange(end, device=freqs.device)  # type: ignore
freqs = torch.outer(t, freqs).float()  # type: ignore
freqs_cos = torch.cos(freqs)  # real part
freqs_sin = torch.sin(freqs)  # imaginary part
freqs_cos, freqs_sin


tensor([1.0000, 0.1000, 0.0100])


(tensor([[ 1.0000,  1.0000,  1.0000],
         [ 0.5403,  0.9950,  0.9999],
         [-0.4161,  0.9801,  0.9998],
         [-0.9900,  0.9553,  0.9996],
         [-0.6536,  0.9211,  0.9992],
         [ 0.2837,  0.8776,  0.9988],
         [ 0.9602,  0.8253,  0.9982],
         [ 0.7539,  0.7648,  0.9976],
         [-0.1455,  0.6967,  0.9968],
         [-0.9111,  0.6216,  0.9960]]),
 tensor([[ 0.0000,  0.0000,  0.0000],
         [ 0.8415,  0.0998,  0.0100],
         [ 0.9093,  0.1987,  0.0200],
         [ 0.1411,  0.2955,  0.0300],
         [-0.7568,  0.3894,  0.0400],
         [-0.9589,  0.4794,  0.0500],
         [-0.2794,  0.5646,  0.0600],
         [ 0.6570,  0.6442,  0.0699],
         [ 0.9894,  0.7174,  0.0799],
         [ 0.4121,  0.7833,  0.0899]]))

In [25]:
B,T,n_head,C = 3,end,4,dim

In [26]:
xk = torch.rand(B,T,n_head,C )
xq = torch.rand(B,T,n_head,C )
xk.shape,xq.shape

(torch.Size([3, 10, 4, 6]), torch.Size([3, 10, 4, 6]))

In [27]:
xq_r, xq_i = xq.float().reshape(xq.shape[:-1] + (-1, 2)).unbind(-1)
xk_r, xk_i = xk.float().reshape(xk.shape[:-1] + (-1, 2)).unbind(-1)

print(xq.float().reshape(xq.shape[:-1] + (-1, 2)).shape)
print(xq.float().reshape(xq.shape[:-1] + (-1, 2)).unbind(-1)[0].shape)

torch.Size([3, 10, 4, 3, 2])
torch.Size([3, 10, 4, 3])


In [28]:
def reshape_for_broadcast(freqs_cis: torch.Tensor, x: torch.Tensor):
    ndim = x.dim()
    assert 1 < ndim
    assert freqs_cis.shape == (x.shape[1], x.shape[-1]), f"{freqs_cis.shape=}, {(x.shape[1], x.shape[-1])=}"
    shape = [d if i == 1 or i == ndim - 1 else 1 for i, d in enumerate(x.shape)] # keep 2nd (T) and last(freq) dim same else make dim 1 for freq_cis
    print(shape)
    return freqs_cis.view(shape)

freq_reshape = reshape_for_broadcast(freqs_cos,xq_r)
freqs_cos.shape, freq_reshape.shape

[1, 10, 1, 3]


(torch.Size([10, 3]), torch.Size([1, 10, 1, 3]))

In [29]:
freqs_cos = reshape_for_broadcast(freqs_cos,xq_r)
freqs_sin = reshape_for_broadcast(freqs_sin,xk_r)


[1, 10, 1, 3]
[1, 10, 1, 3]


In [30]:
xq_out_r = xq_r * freqs_cos - xq_i * freqs_sin # (ac-bd) 
xq_out_i = xq_r * freqs_sin + xq_i * freqs_cos # (ad+bc) * i

In [32]:
xq_out_r.shape

torch.Size([3, 10, 4, 3])